# Model Packaging for GeoImageNet

This notebook explains how to package your model using the thelper library.
First install the required library:

In [11]:
%%bash
pip3 --quiet install  torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python googledrivedownloader pretrainedmodels albumentations pyyaml
pip3 --quiet install affine geojson shapely pyproj hdf5plugin
pip uninstall thelper
rm -rf ./thelper
git clone https://github.com/sfoucher/thelper
pip3 install --quiet -e ./thelper

Can't uninstall 'thelper'. No files were found to uninstall.


Cloning into 'thelper'...


The first time you are installing thelper, you may need to restart the current session

In [ ]:
import os
os.kill(os.getpid(), 9)

Make sure thelper is imported correctly

In [7]:
import os
import torch
import torchvision
import thelper
# initialisation des logs (pour TOUT imprimer ce qui se passe)
thelper.utils.init_logger()

This function below is useful to download datasets from google drive.

In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os.path as osp
import sys


def maybe_download_and_extract(file_id, dest_path ):
    filename = dest_path.split('/')[-1]
    file_path = dest_path
    download_dir= osp.dirname(osp.abspath(dest_path))
    if not os.path.isfile(dest_path):
      gdd.download_file_from_google_drive(file_id= file_id, dest_path= file_path)
      print("Download finished. Extracting files.")

      if file_path.endswith(".zip"):
          # Unpack the zip-file.
          zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
      elif file_path.endswith((".tar.gz", ".tgz")):
          # Unpack the tar-ball.
          tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)
      print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")

We download a Resnet-18 trained on DeepGlobe:

In [3]:
maybe_download_and_extract('1Xl3LYBkmN-MVqBFR5eenHskOem-xVIY9','/content/model.pth')

Download finished. Extracting files.
Done.


### Classfication Task

Specify your class names:

In [2]:
class_names = ["AgriculturalLand", "BarrenLand", "ForestLand", "RangeLand", "UrbanLand", "Water"]

Check that they are defined in the taxonomy

In [5]:
import requests
import json
response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?taxonomy_name=land-cover')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': [{'id': 206, 'name_fr': 'Urbain ou bâti', 'taxonomy_id': 2, 'code': 'URBA', 'name_en': 'Urban or Built-up Land', 'children': [{'id': 207, 'name_fr': 'Zone résidentielle', 'taxonomy_id': 2, 'code': 'RESD', 'name_en': 'Residential', 'children': [{'id': 208, 'name_fr': 'Densité faible', 'taxonomy_id': 2, 'code': 'DENS', 'name_en': 'Low density', 'children': []}, {'id': 209, 'name_fr': 'Densité moyenne', 'taxonomy_id': 2, 'code': 'DENI', 'name_en': 'Medium density', 'children': []}, {'id': 210, 'name_fr': 'Densité élevée', 'taxonomy_id': 2, 'code': 'DENT', 'name_en': 'High density', 'children': []}, {'id': 211, 'name_fr': 'Parc de maisons mobiles', 'taxonomy_id': 2, 'code': 'PARC', 'name_en': 'Mobile home park', 'children': []}]}, {'id': 212, 'name_fr': 'Zone commerciales et services', 'taxonomy_id': 2, 'code': 'COMC', 'name_en': 'Commercial and Services', 'children': [{'id':

Define a class mapping between your class names and the taxonomy ids

In [ ]:
class_mapping = [('AgriculturalLand', 223), ('BarrenLand', 252), ('ForestLand', 233), ('RangeLand', 229),
                  ('UrbanLand', 200), ('Water', 239)]

Define the task as a classification task:

In [3]:
task_config = {
    "type": "thelper.tasks.Classification",
    "params": {
        "class_names": class_names,
        "input_key": "data",
        "label_key": "label"
    }
}

Define the data loader, you'll need to specify the bands you are using (starting with channel no 1):



In [4]:
datasets_config = {
    "deepglobe_test": {
        "type": "thelper.data.geo.ImageFolderGDataset",
        "params": {"root": "/content/",
                    "image_key": "image",
                    "channels": [1,2,3]

                    },
        "task": task_config
    }
}

Define the pre-processing pipeline as following:

In [5]:
loaders_config = {
    #"batch_size": 1,
    "base_transforms": [

        {
            "operation": "thelper.transforms.Resize",
            "params": {"dsize": [224, 224]},
        },
        {
            "operation": "thelper.transforms.NormalizeMinMax",
            "params": {
                "min": [0, 0, 0],
                "max": [255, 255, 255]
            },
        },
        {
            "operation": "thelper.transforms.NormalizeZeroMeanUnitVar",
            "params": {
                "mean": [0.485, 0.456, 0.406],
                "std": [0.229, 0.224, 0.225]
            },
        },
        {
            "operation": "torchvision.transforms.ToTensor",
        },
    ],

    #"test_split": {
    #    "deepglobe_test": 1.0
    #},
}

In [ ]:
Define the model configuration

In [8]:
model_config = {
    "type": torchvision.models.resnet18,
    "params": {"pretrained": True},
    "state_dict": '/content/model.pth',
    "task": task_config
}

In [9]:
config = {"name": 'deepglobe-resnet-18', "model": model_config, "datasets": datasets_config, "loaders": loaders_config}

In [10]:

export_config = {
    "ckpt_name": "test-resnet18-deepglobe.pth",
    "trace_name": "test-resnet18-deepglobe.zip",
    "save_raw": True,
    "trace_input": "torch.rand(1, 3, 224, 224)",
    "task": {
        "type": "thelper.tasks.Classification",
        "params": {
            "class_names": class_names,
            "input_key": "0",
            "label_key": "1"
        }
    }
}


thelper.cli.export_model(config, '/content/thelper-export')



[2020-09-04 20:26:54,883 - thelper.data.utils] DEBUG : loading datasets templates
[2020-09-04 20:26:54,884 - thelper.data.utils] DEBUG : loading dataset 'deepglobe_test' configuration...
[2020-09-04 20:26:55,051 - thelper.data.parsers] WARNING : class name ordering changed due to folder name sorting; this may impact the behavior of previously-trained models as task class indices may be swapped!
[2020-09-04 20:26:55,053 - thelper.data.utils] WARNING : 'task' field detected in dataset 'deepglobe_test' config; dataset's default task will be ignored
[2020-09-04 20:26:55,054 - thelper.cli.export_model] INFO : exporting model 'deepglobe-resnet-18'...
[2020-09-04 20:26:55,061 - thelper.utils.get_save_dir] INFO : output root directory = /content/thelper-export
[2020-09-04 20:26:55,312 - thelper.utils.get_save_dir] INFO : output session directory = /content/thelper-export/deepglobe-resnet-18
[2020-09-04 20:26:55,315 - thelper.utils.get_save_dir] INFO : output logs directory = /content/thelper-e

[2020-09-04 20:27:05,947 - thelper.nn.utils] INFO : reconnecting fc layer for outputting 6 classes...
[2020-09-04 20:27:05,949 - thelper.nn.utils] DEBUG : loading state dictionary from checkpoint into model
[2020-09-04 20:27:05,971 - thelper.nn.utils] DEBUG : previous model task = thelper.tasks.classif.Classification(class_names=['AgriculturalLand', 'BarrenLand', 'ForestLand', 'RangeLand', 'UrbanLand', 'Water'], input_key='data', label_key='label', meta_keys=[])
[2020-09-04 20:27:05,973 - thelper.nn.utils] DEBUG : refreshing model for new task = thelper.tasks.classif.Classification(class_names=['AgriculturalLand', 'BarrenLand', 'ForestLand', 'RangeLand', 'UrbanLand', 'Water'], input_key='data', label_key='label', meta_keys=[])
[2020-09-04 20:27:05,979 - thelper.nn.utils] INFO : module 'torchvision.models.resnet.resnet18' parameter count: 11179590
[2020-09-04 20:27:05,980 - thelper.nn.utils] INFO : ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=

[2020-09-04 20:27:06,271 - thelper.cli.export_model] DEBUG : all done


The code below requires geoimagenet API

In [ ]:

update_model_class_mapping(class_mapping,
                            '/home/sfoucher/DEV/geoimagenet/pth/deepglobe-resnet-18/deepglobe-resnet-18.export.pth', None)
# success, model, buffer, exception = load_model('/home/sfoucher/DEV/geoimagenet/pth/deepglobe-unet/deepglobe-unet.export.pth')
# validate_model(model)